In [1]:
from qiskit import Aer, IBMQ, QuantumCircuit, transpile, QuantumRegister
from qiskit.transpiler import CouplingMap

In [2]:
provider = IBMQ.load_account()

/tmp/ipykernel_13410/2020123530.py:1: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  provider = IBMQ.load_account()
/tmp/ipykernel_13410/2020123530.py:1: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  provider = IBMQ.load_account()


In [3]:
backend = provider.get_backend('ibm_perth')

In [4]:
coupling_map = CouplingMap(backend.configuration().coupling_map)

In [5]:
cm_alice = coupling_map.reduce([0, 1, 2])
cm_bob = coupling_map.reduce([4, 5, 6])

In [18]:
circ_alice = QuantumCircuit(3)
circ_alice.cx([1, 2], [0])
circ_alice.draw()

layout = {circ_alice.qubits[0]:0, circ_alice.qubits[1]:1, circ_alice.qubits[2]:2}

In [19]:
tr_circ_alice = transpile(circ_alice, initial_layout = layout, coupling_map = cm_alice)

In [20]:
tr_circ_alice.draw()

┌───┐   ┌───┐
q_0 -> 0 ┤ X ├───┤ X ├
         └─┬─┘   └─┬─┘
q_1 -> 1 ──■───X───■──
               │      
q_2 -> 2 ──────X──────

In [9]:
tr_circ_bob = tr_circ_alice.copy()

In [10]:
qreg_alice = QuantumRegister(3, 'qr_a')
qreg_bob = QuantumRegister(3, 'qr_b')
qreg_anc = QuantumRegister(1, 'qr_anc')
final_circuit = QuantumCircuit(qreg_alice, qreg_anc, qreg_bob)
final_circuit.compose(tr_circ_alice, qubits = [0, 4, 2], inplace = True)
final_circuit.swap([1, 3], [3, 5])
final_circuit.compose(tr_circ_bob, qubits = [1, 5, 6], inplace = True)


In [11]:
final_circuit.draw()

┌───┐                           
qr_a_0: ┤ X ├────X──────────────────────
        └─┬─┘    │         ┌───┐        
qr_a_1: ──┼───X──┼─────────┤ X ├─X──────
          │   │  │         └─┬─┘ │      
qr_a_2: ──┼───┼──┼──────■────┼───┼──────
          │   │  │      │    │   │      
qr_anc: ──┼───X──┼──X───┼────┼───┼──────
          │      │  │ ┌─┴─┐  │   │      
qr_b_0: ──■──────X──┼─┤ X ├──┼───┼──────
                    │ └───┘  │   │ ┌───┐
qr_b_1: ────────────X────────■───X─┤ X ├
                                   └─┬─┘
qr_b_2: ─────────────────────────────■──

In [12]:
initial_layout = {}
for i, q in enumerate(final_circuit.qubits):
    initial_layout[i] = q

In [13]:
tr_final_circuit = transpile(final_circuit, initial_layout = initial_layout, backend = backend, optimization_level = 0)

In [14]:
tr_final_circuit.draw()

┌───┐                         ┌───┐                         »
  qr_a_0 -> 0 ──■──┤ X ├──■───────────────────■──┤ X ├──■──────────────────────»
              ┌─┴─┐└─┬─┘┌─┴─┐     ┌───┐     ┌─┴─┐└─┬─┘┌─┴─┐     ┌───┐          »
  qr_a_1 -> 1 ┤ X ├──■──┤ X ├──■──┤ X ├──■──┤ X ├──■──┤ X ├──■──┤ X ├──■───────»
              └───┘     └───┘  │  └─┬─┘  │  └───┘     └───┘  │  └─┬─┘  │       »
  qr_a_2 -> 2 ─────────────────┼────┼────┼───────────────────┼────┼────┼───────»
                             ┌─┴─┐  │  ┌─┴─┐┌───┐          ┌─┴─┐  │  ┌─┴─┐     »
qr_anc_0 -> 3 ───────────────┤ X ├──■──┤ X ├┤ X ├──────────┤ X ├──■──┤ X ├──■──»
                   ┌───┐     └───┘     └───┘└─┬─┘          └───┘     └───┘  │  »
  qr_b_0 -> 4 ──■──┤ X ├──■───────────────────┼─────────────────────────────┼──»
              ┌─┴─┐└─┬─┘┌─┴─┐                 │                           ┌─┴─┐»
  qr_b_1 -> 5 ┤ X ├──■──┤ X ├─────────────────■───────────────────────────┤ X ├»
              └───┘     └───┘                                             └───┘»
  qr_b_2 -> 6 ─────────────────────────────────────────────────────────────────»
                                                                               »
«                                            ┌───┐                         »
«  qr_a_0 -> 0 ───────────────────────────■──┤ X ├──■──────────────────────»
«                             ┌───┐     ┌─┴─┐└─┬─┘┌─┴─┐     ┌───┐     ┌───┐»
«  qr_a_1 -> 1 ────────────■──┤ X ├──■──┤ X ├──■──┤ X ├──■──┤ X ├──■──┤ X ├»
«                          │  └─┬─┘  │  └───┘     └───┘  │  └─┬─┘  │  └─┬─┘»
«  qr_a_2 -> 2 ────────────┼────┼────┼───────────────────┼────┼────┼────■──»
«              ┌───┐     ┌─┴─┐  │  ┌─┴─┐               ┌─┴─┐  │  ┌─┴─┐┌───┐»
«qr_anc_0 -> 3 ┤ X ├──■──┤ X ├──■──┤ X ├───────────────┤ X ├──■──┤ X ├┤ X ├»
«              └─┬─┘  │  ├───┤     ├───┤     ┌───┐     └───┘     └───┘└─┬─┘»
«  qr_b_0 -> 4 ──┼────┼──┤ X ├──■──┤ X ├──■──┤ X ├──■───────────────────┼──»
«                │  ┌─┴─┐└─┬─┘┌─┴─┐└─┬─┘┌─┴─┐└─┬─┘┌─┴─┐                 │  »
«  qr_b_1 -> 5 ──■──┤ X ├──■──┤ X ├──■──┤ X ├──■──┤ X ├─────────────────■──»
«                   └───┘     └───┘     └───┘     └───┘                    »
«  qr_b_2 -> 6 ────────────────────────────────────────────────────────────»
«                                                                          »
«                                  
«  qr_a_0 -> 0 ────────────────────
«                                  
«  qr_a_1 -> 1 ────────────────────
«                                  
«  qr_a_2 -> 2 ────────────────────
«                   ┌───┐          
«qr_anc_0 -> 3 ──■──┤ X ├──■───────
«                │  └─┬─┘  │       
«  qr_b_0 -> 4 ──┼────┼────┼───────
«              ┌─┴─┐  │  ┌─┴─┐┌───┐
«  qr_b_1 -> 5 ┤ X ├──■──┤ X ├┤ X ├
«              └───┘     └───┘└─┬─┘
«  qr_b_2 -> 6 ─────────────────■──
«

In [15]:
tr_final_circuit.qubits

[Qubit(QuantumRegister(7, 'q'), 0),
 Qubit(QuantumRegister(7, 'q'), 1),
 Qubit(QuantumRegister(7, 'q'), 2),
 Qubit(QuantumRegister(7, 'q'), 3),
 Qubit(QuantumRegister(7, 'q'), 4),
 Qubit(QuantumRegister(7, 'q'), 5),
 Qubit(QuantumRegister(7, 'q'), 6)]